In [1]:
base_path = "/content/drive/My Drive/Github/mtc-device-activation"

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [3]:
# Load saved data
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append(f'{base_path}/frameworks')

from casbl import casbl

data = np.load(f'{base_path}/general_model/data/general_model.npz', allow_pickle=True)
all_DeviceLocations = data['all_DeviceLocationsUncorrelated']
num_samples = data['num_samples']
device_count = data['device_count']
sparsity = data['sparsity']

data = np.load(f'{base_path}/communication_model/data/communication_simulation_theta.npz', allow_pickle=True)
all_theta = data["all_theta"]

data = np.load(f'{base_path}/communication_model/data/communication_simulation_gamma.npz', allow_pickle=True)
all_gamma = data["all_gamma"]

data = np.load(f'{base_path}/communication_model/data/communication_simulation_z.npz', allow_pickle=True)
all_z = data["all_z"]

data = np.load(f'{base_path}/communication_model/data/communication_simulation_noise.npz', allow_pickle=True)
all_noise = data["all_noise"]

data = np.load(f'{base_path}/communication_model/data/communication_simulation_received_signal.npz', allow_pickle=True)
all_received_signal = data["all_received_signal"]

data = np.load(f'{base_path}/communication_model/data/communication_simulation_general.npz', allow_pickle=True)
snr_db_array = data["snr_db_array"]
pilot_length_array = data["pilot_length_array"]
M = data["M"]

pilot_length_index = 2
pilot_length=pilot_length_array[pilot_length_index]

max_iter = 500

Mounted at /content/drive


In [4]:
sample_range = range(600, 800)
num_samples = len(sample_range)

In [5]:
# Final gamma and z results
gamma_result_all = np.empty((num_samples, len(snr_db_array)), dtype=object)
z_result_all = np.empty((num_samples, len(snr_db_array)), dtype=object)

In [6]:
for sample_index in tqdm(sample_range, desc="Processing Samples", position=0):
    gamma = all_gamma[sample_index]  # Ground truth gamma values (true support)
    loc = all_DeviceLocations[sample_index]
    z = all_z[sample_index]

    for snr_db_index in tqdm(range(len(snr_db_array)), desc="Processing SNR DB Levels", leave=False, position=1):
        snr_db = snr_db_array[snr_db_index]

        theta = all_theta[sample_index, pilot_length_index, :pilot_length]
        received_signal = all_received_signal[sample_index, pilot_length_index, snr_db_index, :pilot_length]

        # Compute noise
        snr = 10 ** (snr_db / 10)
        signal_power = np.mean(np.abs(np.matmul(theta, z))**2)
        noise_power = signal_power / snr

        # Run CASBL
        gamma_result, z_result, gamma_history, z_history, iteration_count = casbl(
            theta, received_signal, noise_power, loc, max_iter=max_iter
        )

        # Save outputs
        gamma_result_all[sample_index-sample_range[0], snr_db_index] = gamma_result.copy()
        z_result_all[sample_index-sample_range[0], snr_db_index] = z_result.copy()

Processing Samples:   0%|          | 1/200 [01:06<3:40:13, 66.40s/it]

Converged after 346 iterations



Processing SNR DB Levels:  83%|████████▎ | 5/6 [01:00<00:12, 12.06s/it]

Converged after 467 iterations



Processing Samples:   1%|          | 2/200 [02:19<3:52:27, 70.44s/it]

...

Converged after 499 iterations



Processing Samples: 100%|█████████▉| 199/200 [3:51:25<01:12, 72.71s/it]

Converged after 434 iterations



Processing Samples: 100%|██████████| 200/200 [3:52:38<00:00, 69.79s/it]

Converged after 413 iterations


In [7]:
np.savez_compressed(
    f'{base_path}/estimation/snr_db_all_pilot_length_30/data/casbl_uc_snr_db_all_pilot_length_30_set_4.npz',
    gamma_result_all=gamma_result_all,
    z_result_all=z_result_all,
)

print("All results have been saved")

All results have been saved
